In [ ]:
# Environment: genai_data (py3.10) | datasets==2.14.6 | pyarrow==12.0.1 | numpy<2

import os, sys, multiprocessing, warnings, pandas as pd
warnings.filterwarnings("ignore")

print("Python:", sys.version.split()[0])
print("CPU Cores Available:", multiprocessing.cpu_count())

# Create folders
os.makedirs("../data/cleaned", exist_ok=True)
print("Folder ready → data/cleaned (raw data cached by Hugging Face automatically)")


In [ ]:
!pip install -q "numpy<2.0.0" "datasets==2.14.6" "pyarrow==12.0.1" pandas matplotlib seaborn tqdm

from datasets import get_dataset_config_names, load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from glob import glob

plt.style.use("seaborn-v0_8")
sns.set_theme(context="talk", style="whitegrid", palette="crest")


In [ ]:
DATASET_ID = "McAuley-Lab/Amazon-Reviews-2023"
CLEAN_DIR = "data/cleaned"
MIN_PRICE, MAX_PRICE = 0.5, 999.49
MAX_WORKERS = 8   # 

print(f"Dataset → {DATASET_ID}")
print(f"Using {MAX_WORKERS} CPU workers")

cols_to_keep = [
    "main_category",
    "title",
    "features",
    "description",
    "average_rating",
    "rating_number",
    "store",
    "category",
    "price_num"
]



In [ ]:
def safe_float(x):
    try:
        return float(x)
    except Exception:
        return None


def clean_subset(subset_name: str):
    """Clean one subset and save as Parquet + CSV."""
    try:
        ds = load_dataset(DATASET_ID, subset_name, split="full")  
        df = pd.DataFrame(ds)
        df["category"] = subset_name.replace("raw_meta_", "")
        df["price_num"] = pd.to_numeric(df["price"], errors="coerce")

        # Apply filters
        df = df.dropna(subset=["price_num"])
        df = df[(df["price_num"] >= MIN_PRICE) & (df["price_num"] <= MAX_PRICE)]
        df = df[df["title"].astype(str).str.strip() != ""]
        df = df[df["description"].apply(lambda x: isinstance(x, list) and len(x) > 0)]

        # Save to both formats
        base = os.path.join(CLEAN_DIR, subset_name.replace("raw_meta_", ""))
        df.to_parquet(base + "_clean.parquet", index=False)
        df.to_csv(base + "_clean.csv", index=False)

        return subset_name, len(df)
    except Exception as e:
        return subset_name, f"Error: {e}"


In [ ]:
print(f"Fetching subset names for {DATASET_ID} …")
subsets = get_dataset_config_names(DATASET_ID)
meta_subsets = [s for s in subsets if s.startswith("raw_meta_")]
print(f"Found {len(meta_subsets)} metadata subsets.\n")

results = []
with ProcessPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = [ex.submit(clean_subset, s) for s in meta_subsets]
    for f in tqdm(as_completed(futures), total=len(futures), desc="🧹 Cleaning Subsets"):
        results.append(f.result())

print("\nCleaning Summary:")
for subset, res in results:
    print(f"  • {subset}: {res}")


In [ ]:
import pyarrow.parquet as pq
import os
from glob import glob

# Path to all cleaned parquet files
parquet_files = sorted(glob(os.path.join("data/cleaned", "*_clean.parquet")))

columns_dict = {}

print(f"Inspecting {len(parquet_files)} cleaned subsets...\n")

for file_path in parquet_files:
    try:
        table = pq.read_table(file_path)
        columns = table.column_names
        columns_dict[os.path.basename(file_path)] = columns
        print(f"{os.path.basename(file_path)} → {len(columns)} columns")
    except Exception as e:
        print(f"{os.path.basename(file_path)} → {e}")

print("\n Summary:")
print(f"Total files scanned: {len(columns_dict)}")

# Save the dictionary to a JSON for later inspection
import json
with open("data/cleaned/columns_summary.json", "w") as f:
    json.dump(columns_dict, f, indent=2)

print("\nColumns summary saved → data/cleaned/columns_summary.json")


In [ ]:
parquet_files = sorted(glob(os.path.join(CLEAN_DIR, "*_clean.parquet")))
print(f"Merging {len(parquet_files)} cleaned subsets with selected columns ...")

tables = []
for path in parquet_files:
    try:
        schema = pq.read_schema(path)
        cols_in_file = [c for c in cols_to_keep if c in schema.names]
        if not cols_in_file:
            print(f" Skipping {os.path.basename(path)} → No matching columns")
            continue

        # Read table
        t = pq.read_table(path, columns=cols_in_file)

        # Harmonize data types to avoid merge conflict
        for name in t.schema.names:
            if name in ["price_num", "average_rating", "rating_number"]:
                # Always cast numeric fields to float64
                t = t.set_column(t.schema.get_field_index(name),
                                 name,
                                 pa.compute.cast(t[name], pa.float64()))
        tables.append(t)

    except Exception as e:
        print(f"Skipping {os.path.basename(path)} → {e}")

# Step 2 – Combine Tables Safely
if tables:
    combined = pa.concat_tables(tables, promote=True)
    print(f"Combined Arrow table: {combined.num_rows:,} rows, {len(combined.column_names)} columns")
else:
    raise ValueError(" tables were loaded successfully! Check your cleaned folder.")

# Step 3 – Save Master Parquet
final_parquet = os.path.join(CLEAN_DIR, "amazon_reviews_2023_selected.parquet")
pq.write_table(combined, final_parquet, compression="snappy")
print(f"Saved master Parquet → {final_parquet}")

In [ ]:
# Convert to Pandas for visualization (only lightweight columns)
df_all = combined.select([c for c in cols_to_keep if c != "description" and c != "features"]).to_pandas()
print(f"Final DataFrame shape: {df_all.shape}")

# Optional: Save lightweight CSV (for human inspection)
final_csv = os.path.join(CLEAN_DIR, "amazon_reviews_2023_selected.csv")
df_all.to_csv(final_csv, index=False)
print(f"Saved CSV (light, no nested fields) → {final_csv}")


In [ ]:
print("Basic Statistics:")

# Only show stats for numeric columns if they exist
numeric_cols = [col for col in ["price_num", "average_rating", "rating_number"] if col in df_all.columns]
if numeric_cols:
    display(df_all[numeric_cols].describe().T)
else:
    print(" No numeric columns found for statistical summary.")

print("\n Top 10 Categories by Count:")

if "category" in df_all.columns:
    top_categories = df_all["category"].value_counts().head(10)
    display(top_categories)

    # Optional barplot visualization for top categories
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_categories.values, y=top_categories.index, orient="h", palette="crest")
    plt.title(" Top 10 Product Categories by Count", fontsize=16, fontweight="bold")
    plt.xlabel("Number of Products")
    plt.ylabel("Category")
    plt.tight_layout()
    plt.show()
else:
    print(" Column 'category' not found in dataframe.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker

#  Unified High-Contrast Style

sns.set_theme(context="talk", style="whitegrid")
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "axes.edgecolor": "black",
    "axes.linewidth": 1.3,
    "grid.alpha": 0.25,
    "axes.labelweight": "bold",
    "axes.labelsize": 13,
    "axes.titlesize": 16,
    "legend.fontsize": 12,
    "font.weight": "medium"
})

palette = sns.color_palette("coolwarm", as_cmap=False)

In [ ]:

# 1️Price Distribution (Log Scale)

plt.figure(figsize=(12, 6))
sns.histplot(df_all["price_num"], bins=80, kde=True, color=palette[0], edgecolor="black", alpha=0.8)
plt.xscale("log")
plt.title(" Price Distribution (Log Scale)", fontsize=18, fontweight="bold", pad=15)
plt.xlabel("Product Price ($, log scale)")
plt.ylabel("Number of Products")
plt.tight_layout()
plt.show()

In [ ]:

# 2 Average Rating Distribution

plt.figure(figsize=(12, 6))
sns.histplot(df_all["average_rating"].dropna(), bins=30, kde=True, color=palette[-1], edgecolor="black", alpha=0.85)
plt.title("Average Rating Distribution", fontsize=18, fontweight="bold", pad=15)
plt.xlabel("Average Rating")
plt.ylabel("Number of Products")
plt.xlim(0.5, 5.5)
plt.tight_layout()
plt.show()

In [ ]:

# 3️Top Categories by Count (Cleaned & Spaced)
if "category" in df_all.columns:
    top_cats = df_all["category"].value_counts().head(15)
    plt.figure(figsize=(12, 7))
    sns.barplot(x=top_cats.values, y=top_cats.index, palette="viridis", edgecolor="black", alpha=0.9)
    plt.title("Top 15 Product Categories by Count", fontsize=18, fontweight="bold", pad=15)
    plt.xlabel("Number of Products")
    plt.ylabel("Category")

    # Limit x-axis labels to only first, middle, last for clarity
    ax = plt.gca()
    ticks = ax.get_xticks()
    if len(ticks) > 3:
        spaced_ticks = [ticks[0], ticks[len(ticks)//3], ticks[2*len(ticks)//3], ticks[-1]]
        ax.set_xticks(spaced_ticks)
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x):,}"))
    plt.grid(True, axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()


In [ ]:

#  Average Price per Category
if {"price_num", "category"} <= set(df_all.columns):
    avg_price = (
        df_all.groupby("category")["price_num"]
        .mean()
        .sort_values(ascending=False)
        .head(15)
    )

    plt.figure(figsize=(12, 7))
    sns.barplot(x=avg_price.values, y=avg_price.index, palette="mako", edgecolor="black", alpha=0.9)
    plt.title("Average Price per Category (Top 15)", fontsize=18, fontweight="bold", pad=15)
    plt.xlabel("Average Price ($)")
    plt.ylabel("Category")
    plt.grid(True, axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()